In [67]:
"""
#This script demonstrates the use of a convolutional LSTM network.

This network is used to predict the next frame of an artificially
generated movie which contains moving squares.
"""
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import numpy as np
import pylab as plt

In [68]:
# We create a layer which take as input movies of shape
# (n_frames, width, height, channels) and returns a movie
# of identical shape.
imgSize = 16
seq = Sequential()
seq.add(ConvLSTM2D(filters=128, kernel_size=(5, 5),
                   input_shape=(None, imgSize, imgSize, 16),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=64, kernel_size=(5, 5),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=64, kernel_size=(5, 5),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

# seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
#                    padding='same', return_sequences=True))
# seq.add(BatchNormalization())

seq.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
               activation='sigmoid',
               padding='same', data_format='channels_last'))
seq.compile(loss='binary_crossentropy', optimizer='adadelta')

In [69]:
seq.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_1 (ConvLSTM2D)  (None, None, 16, 16, 128) 1843712   
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 16, 16, 128) 512       
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, None, 16, 16, 64)  1229056   
_________________________________________________________________
batch_normalization_2 (Batch (None, None, 16, 16, 64)  256       
_________________________________________________________________
conv_lst_m2d_3 (ConvLSTM2D)  (None, None, 16, 16, 64)  819456    
_________________________________________________________________
batch_normalization_3 (Batch (None, None, 16, 16, 64)  256       
_________________________________________________________________
conv3d_1 (Conv3D)            (None, None, 16, 16, 1)   1729      
Total para

In [ ]:
a = np.load('mnist_test_seq.npy')
a = np.swapaxes(a, 0, 1)
print(a.shape)
b = np.zeros((10000,20,16,16,16))
for i in range(16):
    for j in range(16):
        b[:, :, i, j, :] = a[:, :, i::16, j::16].reshape((10000, 20, -1))
del a

(10000, 20, 64, 64)


In [55]:
# a = np.arange(0,16).reshape((4, 4))
# print(a)
# b = np.zeros((2, 2, 4))
# for i in range(2):
#     for j in range(2):
#         b[i, j, :] = a[i::2, j::2].flatten()
# print(b[:, :, 0])

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
[[0. 1.]
 [4. 5.]]


In [1]:
noisy_movies = b[:1200, :15, :, :, :]/255.0
shifted_movies = b[:1200, 1:16, :, :, :]/255.0

NameError: name 'np' is not defined

In [8]:
# import matplotlib.pyplot as pyplt
# %matplotlib inline
# print(noisy_movies.shape)
# for i in range(3):
#     pyplt.imshow(noisy_movies[0, i, :, :, 0])
#     pyplt.show()
#     pyplt.imshow(shifted_movies[0, i, :, :, 0])
#     pyplt.show()

In [9]:
# def data_gen(features, labels, batch_size):
#  # Create empty arrays to contain batch of features and labels#
#     while True:
#         for i in np.arange(0, features.shape[0] - batch_size, batch_size):
#             # choose random index in features
#             batch_features = features[i:i+batch_size,:, :, :, :]
#             batch_labels= labels[i:i+batch_size, :, :, :, :]
#             yield (batch_features, batch_labels)

In [5]:
# seq.fit_generator(data_gen(noisy_movies[:1000], shifted_movies[:1000], 10),
#                   steps_per_epoch = 100, epochs = 20,
#                     validation_data=data_gen(noisy_movies[-200:-1], shifted_movies[-200:-1], 5), 
#                     validation_steps = 5,verbose=1)
seq.fit(noisy_movies[:1000], shifted_movies[:1000], batch_size=10,
        epochs=30, validation_split=0.05)        

Instructions for updating:
Use tf.cast instead.
Train on 950 samples, validate on 50 samples
Epoch 1/30
620/950 [==================>...........] - ETA: 2:33 - loss: 0.3479

KeyboardInterrupt: 

In [ ]:
# Testing the network on one movie
# feed it with the first 7 positions and then
# predict the new positions
which = 1004
track = noisy_movies[which][:7, ::, ::, ::]
print(track.shape)

for j in range(16):
    new_pos = seq.predict(track[np.newaxis, ::, ::, ::, ::])
    new = new_pos[::, -1, ::, ::, ::]
    track = np.concatenate((track, new), axis=0)


# And then compare the predictions
# to the ground truth
track2 = noisy_movies[which][::, ::, ::, ::]
for i in range(15):
    fig = plt.figure(figsize=(10, 5))

    ax = fig.add_subplot(121)

    if i >= 7:
        ax.text(1, 3, 'Predictions !', fontsize=20, color='w')
    else:
        ax.text(1, 3, 'Initial trajectory', fontsize=20)

    toplot = track[i, ::, ::, 0]

    plt.imshow(toplot)
    ax = fig.add_subplot(122)
    plt.text(1, 3, 'Ground truth', fontsize=20)

    toplot = track2[i, ::, ::, 0]
    if i >= 2:
        toplot = shifted_movies[which][i - 1, ::, ::, 0]

    plt.imshow(toplot)
    plt.savefig('%i_animate.png' % (i + 1))